In [54]:
%reset
from sympy import *
import numpy as np

Txx, Tyy, Txy = symbols('Txx, Tyy, Txy ')
Exx,Eyy,Exy   = symbols('Exx,Eyy,Exy')
C_pwl, n_pwl  = symbols('C_pwl, n_pwl')
C_lin, n_lin, m_lin  = symbols('C_lin, n_lin, m_lin')
eta_0,eta_el  = symbols('eta_0,eta_el')
dg = symbols('dg') # grain size

# The new guys for anisotropy
d0, d1, ani, two, a = symbols('d0, d1, ani, two, a')
f_ani         = symbols('f_ani')

Tzz      = -Txx-Tyy 
Ezz      = -Exx-Eyy 

Tii       = sqrt(1/2*(Txx**2+Tyy**2+Tzz**2) + a**2*Txy**2)
Jii           = Tii**2
Eii       = sqrt(1/2*(Exx**2+Eyy**2+Ezz**2) + a**2*Exy**2)


# Gzz = symbols('Gzz')
C_ANI    = Matrix([ [-d0, d0, two*d1], [d0, -d0, -two*d1], [d1, -d1, -two*(1/2-d0)]]) ##### !!! - sign in D33 -a0
C_ISO    = Matrix([ [1, 0, 0], [0, 1, 0], [0, 0, two*1/2]])
Dani     = 2* (C_ISO + C_ANI * ani)
E        = Matrix([[Exx],[Eyy],[Exy]])
Gij      = Matrix([ [1/2, 0, 0], [0, 1/2, 0], [0, 0, 1/two]]) * Dani*E
Gxx, Gyy, Gxy = Gij[0], Gij[1], Gij[2]
Gzz      = -Gxx-Gyy 
Gii       = sqrt(1/2*(Gxx**2+Gyy**2+Gzz**2) + a**2*Gxy**2)


In [57]:

#---------------------- VISCO-ELASTIC VISCOSITY ----------------------#

# 1. Viscosity expression as function of stress invariant
eta_pwl       = f_ani**(n_pwl-1)*(2*C_pwl)**(-1)*Tii**(1-n_pwl)
eta_lin       = f_ani**(n_lin-1)*(2*C_lin)**(-1)*Tii**(1-n_lin) * dg**(m_lin)
eta_ve  = (eta_el**-1 + eta_pwl**-1 + eta_lin**-1)**-1
eta_ve  = (eta_el**-1 + eta_pwl**-1 )**-1
print('eta_ve = ' + octave_code(eta_ve.subs(eta_pwl,'eta_pwl').subs(f_ani,'f_ani')) + ';')
# 2. Derivatives of viscosity w.r.t. strain rate components
detadexx = symbols('detadexx')
detadtxx = symbols('detadtxx')
f        = detadexx - ( detadtxx * (2*eta_0 + 2*detadexx*Exx))
detadexx = solve(f,detadexx)[0]
display(detadexx)
print('detadexx = ' + octave_code(detadexx.subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani,'f_ani')) + ';')
# 3. Derivatives of viscosity w.r.t. stress components
detadtxx = eta_ve.diff(Txx)
detadtyy = eta_ve.diff(Tyy)
detadtxy = eta_ve.diff(Txy)
detadtxx = detadtxx.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(eta_ve,'eta_pwl').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Txx),'dfanidTxx').subs(f_ani,'f_ani').simplify()
detadtyy = detadtyy.subs(eta_ve,'eta_ve').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Tyy),'dfanidTyy').subs(f_ani,'f_ani').simplify()
detadtxy = detadtxy.subs(eta_ve,'eta_ve').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Txy),'dfanidTxy').subs(f_ani,'f_ani').simplify()

display(detadtxx)
display(detadtyy)
display(detadtxy)
# for i in range(1):
#     print('detadTxx = ' + ccode(detadtxx) + ';')
#     print('detadTyy = ' + ccode(detadtyy) + ';')
#     print('detadTxy = ' + ccode(detadtxy) + ';')
for i in range(1):
    print('detadTxx = ' + octave_code(detadtxx) + ';')
    print('detadTyy = ' + octave_code(detadtyy) + ';')
    print('detadTxy = ' + octave_code(detadtxy) + ';')


eta_ve = 1./(2*C_pwl.*f_ani.^(1 - n_pwl).*(0.5*Txx.^2 + Txy.^2.*a.^2 + 0.5*Tyy.^2 + 0.5*(-Txx - Tyy).^2).^(n_pwl/2 - 1/2) + 1./eta_el);


-2*detadtxx*eta_0/(2*Exx*detadtxx - 1)

detadexx = -2*detadtxx.*eta_0./(2*Exx.*detadtxx - 1);


-C_pwl*Jii**(n_pwl/2 - 3/2)*eta_ve**2*f_ani**(1 - n_pwl)*(2.0*Txx + 1.0*Tyy)*(n_pwl - 1)

-C_pwl*Jii**(n_pwl/2 - 3/2)*eta_ve**2*f_ani**(1 - n_pwl)*(1.0*Txx + 2.0*Tyy)*(n_pwl - 1)

-2*C_pwl*Jii**(n_pwl/2 - 3/2)*Txy*a**2*eta_ve**2*f_ani**(1 - n_pwl)*(n_pwl - 1)

detadTxx = -C_pwl.*Jii.^(n_pwl/2 - 3/2).*eta_ve.^2.*f_ani.^(1 - n_pwl).*(2.0*Txx + 1.0*Tyy).*(n_pwl - 1);
detadTyy = -C_pwl.*Jii.^(n_pwl/2 - 3/2).*eta_ve.^2.*f_ani.^(1 - n_pwl).*(1.0*Txx + 2.0*Tyy).*(n_pwl - 1);
detadTxy = -2*C_pwl.*Jii.^(n_pwl/2 - 3/2).*Txy.*a.^2.*eta_ve.^2.*f_ani.^(1 - n_pwl).*(n_pwl - 1);


In [40]:
f_ani     = Gii/Eii
dfanidExx = f_ani.diff(Exx).subs(f_ani, 'f_ani').subs(Eii, 'Eii').subs(Gii, 'Gii').subs(Gxx,'Gxx').subs(Gyy,'Gyy').subs(Gxy,'Gxy').subs(Gzz,'Gzz')
dfanidEyy = f_ani.diff(Eyy).subs(f_ani, 'f_ani').subs(Eii, 'Eii').subs(Gii, 'Gii').subs(Gxx,'Gxx').subs(Gyy,'Gyy').subs(Gxy,'Gxy').subs(Gzz,'Gzz')
dfanidExy = f_ani.diff(Exy).subs(f_ani, 'f_ani').subs(Eii, 'Eii').subs(Gii, 'Gii').subs(Gxx,'Gxx').subs(Gyy,'Gyy').subs(Gxy,'Gxy').subs(Gzz,'Gzz')

# display(dfanidExx.subs('f_ani',1.0).subs(ani,0.0).subs(d0,0.0).subs(d1,1.0).subs('Gxx',Exx).subs('Gii','Eii'))
print('dfanidExx = ' + octave_code(dfanidExx) + ';')
print('dfanidEyy = ' + octave_code(dfanidEyy) + ';')
print('dfanidExy = ' + octave_code(dfanidExy) + ';')
# display(f_ani.subs(Gii, 'Gii').subs(ani,0.0))


dfanidExx = (0.5*Exx.*ani.*d0 + 0.5*Exx.*(-1.0*ani.*d0 + 1.0) + 0.5*Eyy.*ani.*d0 + 0.5*Eyy.*(-1.0*ani.*d0 + 1.0) + 0.25*Gxx.*(-2.0*ani.*d0 + 2.0) + 2*Gxy.*a.^2.*ani.*d1./two + 0.5*Gyy.*ani.*d0)./(Eii.*Gii) - 0.5*Exx.*f_ani./Eii.^2;
dfanidEyy = (0.5*Exx.*ani.*d0 + 0.5*Exx.*(-1.0*ani.*d0 + 1.0) + 0.5*Eyy.*ani.*d0 + 0.5*Eyy.*(-1.0*ani.*d0 + 1.0) + 0.5*Gxx.*ani.*d0 - 2*Gxy.*a.^2.*ani.*d1./two + 0.25*Gyy.*(-2.0*ani.*d0 + 2.0))./(Eii.*Gii) - 0.5*Eyy.*f_ani./Eii.^2;
dfanidExy = (0.5*Gxx.*ani.*d1.*two + Gxy.*a.^2.*(-2*ani.*two.*(0.5 - d0) + two)./two - 0.5*Gyy.*ani.*d1.*two)./(Eii.*Gii) - Exy.*a.^2.*f_ani./Eii.^2;


In [66]:
# For grain size evolution one needs to make the viscosity of linear creep stress dependent
df = Function('df')(Txx,Tyy,Txy) # grain size

# 1. Viscosity expression as function of stress invariant
eta_lin       = f_ani**(n_lin-1)*(2*C_lin)**(-1)*Tii**(1-n_lin) * df**(m_lin)
eta_ve  = (eta_el**-1 + eta_lin**-1)**-1

# 3. Derivatives of viscosity w.r.t. stress components
detadtxx = eta_ve.diff(Txx)
detadtyy = eta_ve.diff(Tyy)
detadtxy = eta_ve.diff(Txy)
detadtxx = detadtxx.subs(eta_ve,'eta_ve').subs(eta_ve,'eta_lin').subs(eta_ve,'eta_pwl').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Txx),'dfanidTxx').subs(f_ani,'f_ani').subs(df.diff(Txx),'ddgdTxx').subs(df,'dg').simplify()
detadtyy = detadtyy.subs(eta_ve,'eta_ve').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Tyy),'dfanidTyy').subs(f_ani,'f_ani').simplify()
detadtxy = detadtxy.subs(eta_ve,'eta_ve').subs(1/2*Txx**2 + Txy**2*a**2 + 1/2*Tyy**2 + 1/2*Tzz**2,'Jii').subs(f_ani.diff(Txy),'dfanidTxy').subs(f_ani,'f_ani').simplify()

display(detadtxx)

1.0*C_lin*Jii**(n_lin/2 - 3/2)*dg**(-m_lin - 1)*eta_ve**2*f_ani**(1 - n_lin)*(2.0*Jii*ddgdTxx*m_lin - 2.0*Txx*dg*n_lin + 2.0*Txx*dg - 1.0*Tyy*dg*n_lin + 1.0*Tyy*dg)

In [75]:
# Evaluate sensitivity of grain size to tau
Ared,dg0,dt,Agro,p = symbols('Ared,dg0,dt,Agro,p')
Eii_pwl   = f_ani**(-n_pwl) * C_pwl * Tii**(n_pwl);        
Wpwl      = f_ani*Eii_pwl/Tii*( Exx/Gxx*Txx**2 + Eyy/Gyy*Tyy**2 + Ezz/Gzz*Tzz**2 + 2*Exy/Gxy*Txy**2)
dred      =-Wpwl*dg**2*Ared
dgro      = Agro*dg**(1-p)
dg1 = (dg0) + dt*(dred + dgro)